In [10]:
from treelib import Node, Tree
from stanfordcorenlp import StanfordCoreNLP

# Tree linearization
def linearize(token, dependency_tree):
	output = []
	tmp_dt = sorted(dependency_tree, key=lambda x : x[2])
	for dt in tmp_dt:
		tmp = []
		tmp += [token[dt[2]-1]]
		tmp += [dt[0]]
		tmp += [dt[1]]
		output.append(tmp)
	return output

def store_recursion(tree, n, L, o):
    global global_i
    if o == len(L):
        n.data.append([L[j][2] for j in range(len(L))])
        return
    children_list = tree.children(n.identifier)
    new_n = Node()
    if children_list == []:
        new_n = tree.create_node(tag=L[o][0], identifier=global_i+1, data=[L[o][1]], parent=n.identifier)
        global_i += 1
    else:
        has_child = False
        child_i = 0
        while has_child == False and child_i < len(children_list):
            if children_list[child_i].tag == L[o][0] and children_list[child_i].data[0] == L[o][1]:
                has_child = True
                new_n = children_list[child_i]
                break
            child_i += 1
        if has_child == False:
            new_n = tree.create_node(tag=L[o][0], identifier=global_i+1, data=[L[o][1]], parent=n.identifier)
            global_i += 1
    store_recursion(tree, new_n, L, o+1)

# Return -1 if no such child, return child id otherwise
def has_child(tree, n, Li):
    children_list = tree.children(n)
    child_id = -1
    for i in range(len(children_list)):
        if children_list[i].tag == Li[0] and children_list[i].data[0] == Li[1]:
            child_id = children_list[i].identifier
            return child_id
    return child_id

def is_compatible(L, S, P):
    subseq = []
    for p in P:
        subseq.append(L[p][2])
    return S == subseq

def lookup_recursive(treeI, L, o, n, P, M):
    for i in range(o, len(L)):
        child_id = has_child(treeI, n, L[i])
        if child_id != -1:
            new_n = child_id
            P.add(i)
            if len(treeI.get_node(new_n).data) > 1:
                for j in range(1, len(treeI.get_node(new_n).data)):
                    S = treeI.get_node(new_n).data[j]
                    if is_compatible(L, S, P):
                        new_m = []
                        for p in P:
                            new_m.append(L[p][0])
                        M.append(new_m)
            lookup_recursive(treeI, L, o+1, new_n, P, M)

In [ ]:
#################################################
################# Main Function #################
#################################################
nlp = StanfordCoreNLP(r'/home/xiaodan/Documents/stanford-corenlp-full-2018-02-27')   # Replace with your path
tree = Tree()
root = tree.create_node(tag='Root', identifier=0)#root node
global_i = 0
# Read from file
for i in range(40):
    filenamedir = "/home/xiaodan/Downloads/rephrase/"
    filename = filenamedir+str(i)+'.txt'
    f = open(filename, "r")

    rawtesttext=f.read().strip().split('\n')
    line_count = len(rawtesttext)
    # for line in f:
    #     line_count += 1
    #     rawtesttext.append(line.strip('\n'))
    f.close()

    # Build dependency tree using the stanford tool
    testtext = []
    for text in rawtesttext:
        #text = text[:-1]
        token = nlp.word_tokenize(text)
#         print(token)
        dp = nlp.dependency_parse(text)
#         print(dp)

        output = linearize(token, dp)
        testtext.append(output)
#         print(output)
#         print("_"*30)
    for text in testtext:
        store_recursion(tree, root, text, 0)

# Visualize the tree-trie
tree.show()

In [13]:
Global_M=[]
for i in range(40,50):
    filenamedir = "/home/xiaodan/Downloads/rephrase/"
    filename = filenamedir+str(i)+'.txt'
    f = open(filename, "r")

    rawtesttext=f.read().strip().split('\n')
    line_count = len(rawtesttext)
    f.close()

    # Build dependency tree using the stanford tool
    testtext = []
    for text in rawtesttext:
        token = nlp.word_tokenize(text)
        dp = nlp.dependency_parse(text)
        output = linearize(token, dp)
        testtext.append(output)
    for text in testtext:
        M = []
        P = set()
        lookup_recursive(tree, text, 0, 0, P, M)
        Global_M.append(M)

# Print the match results
print(Global_M)
nlp.close()